## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [16]:
%%sql
postgresql:///animal_shelters

'Connected: @animal_shelters'

In [17]:
%%sql
postgresql:///animal_shelters
    WITH ts_date AS(
        SELECT animalid AS id,
          CAST( DATE_PART('year',AGE( timestamp '2021-12-31' , CAST(CONCAT(RIGHT(birthdate, 4),'-', LEFT(birthdate,2),'-',SUBSTRING(birthdate,4,2 ) ) as timestamp))) as CHAR(2)) 
        AS age
        FROM animals
    ),
    
    ani_size AS (
        SELECT a.animalid AS id, 
        CASE WHEN a.animaltype = 'Dog' THEN
                CASE WHEN a.weight <= 10 THEN 'small'
                WHEN a.weight >10 AND a.weight<=30 THEN 'medium'
                WHEN a.weight >30 THEN 'large' END
             WHEN a.animaltype = 'Cat' THEN
                CASE WHEN a.weight <= 5 THEN 'small'
                    WHEN a.weight >5 AND a.weight<=7 THEN 'medium'
                    WHEN a.weight >7 THEN 'large' END
             WHEN animaltype = 'Bird' THEN
                CASE WHEN a.weight <= 0.7 THEN 'small'
                    WHEN a.weight >0.7 AND a.weight<=1.1 THEN 'medium'
                    WHEN a.weight >1.1 THEN 'large' END
            END AS size
        FROM animals AS a
    ),
     ani_size_code AS(
        SELECT a.animalid AS id,
        CASE
            WHEN a.animaltype = 'Dog' THEN
                CASE WHEN a_s.size='small' THEN 'DS'
                WHEN a_s.size='medium' THEN 'DM'
                WHEN a_s.size='large' THEN 'DL' END
             WHEN a.animaltype = 'Cat' THEN
                CASE WHEN a_s.size='small' THEN 'CS'
                    WHEN a_s.size='medium' THEN 'CM'
                    WHEN a_s.size='large' THEN 'CL' END
             WHEN a.animaltype = 'Bird' THEN
                CASE WHEN a_s.size='small' THEN 'BS'
                    WHEN a_s.size='medium' THEN 'BM'
                    WHEN a_s.size='large' THEN 'BL' END
            END AS code
                    
        FROM ani_size AS a_s, animals AS a
        WHERE a.animalid=a_s.id),
        
    total_cost AS(
        SELECT 
             a.animalid AS id,
            CASE WHEN sa.sponsorid IS NOT NULL THEN 0
                ELSE sc.costs+ac.costs+lc.costs END AS total
            
        FROM animals AS a
        LEFT JOIN sponsored_animals AS sa
                ON a.animalid=sa.sponsorid
        INNER JOIN location_costs AS lc
                on a.location = lc.location
        INNER JOIN ani_size
                ON a.animalid = ani_size.id
        INNER JOIN ani_size_code AS a_s_c
                ON a.animalid = a_s_c.id
        INNER JOIN ts_date AS ts
                ON a.animalid=ts.id
        LEFT JOIN age_costs AS ac
                ON ts.age = ac.age
        INNER JOIN size_costs AS sc
            ON a_s_c.code = sc.sizeid
    )
    
    
    --main code is here
    SELECT  a.animaltype, 
            ani_size.size,
            SUM (tc.total) AS total,
            ROUND((SUM (tc.total)::decimal/(SELECT SUM(total) FROM total_cost))*100,2) AS percentage
        
    FROM animals AS a
        LEFT JOIN sponsored_animals AS sa
                ON a.animalid=sa.sponsorid
        INNER JOIN location_costs AS lc
                on a.location = lc.location
        INNER JOIN ani_size
                ON a.animalid = ani_size.id
        INNER JOIN ani_size_code AS a_s_c
                ON a.animalid = a_s_c.id
        INNER JOIN ts_date AS ts
                ON a.animalid=ts.id
        LEFT JOIN age_costs AS ac
                ON ts.age = ac.age
        INNER JOIN size_costs AS sc
            ON a_s_c.code = sc.sizeid
        INNER JOIN total_cost AS tc
            ON a.animalid=tc.id
 
    GROUP BY a.animaltype, ani_size.size
    ORDER BY a.animaltype, size DESC
   
    

9 rows affected.


animaltype,size,total,percentage
Bird,small,1615,0.05
Bird,medium,3130,0.09
Bird,large,8120,0.23
Cat,small,518015,14.90
Cat,medium,250575,7.21
Cat,large,439490,12.64
Dog,small,336530,9.68
Dog,medium,941895,27.09
Dog,large,977665,28.12
